In [16]:
import numpy as np
import pandas as pd
import cv2
import PIL.Image as Image
import os
import matplotlib.pyplot as plt
import pathlib
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D , MaxPooling2D,Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import model_from_json
import pickle
print("all set !")


all set !


In [17]:
current_dir = os.getcwd()
image_path = os.path.join(current_dir, "data", "img")


In [18]:
path_dict = {"cats": [], "dogs": []}
for dir_name in path_dict.keys():
    directory_path = os.path.join(image_path, dir_name)
    if os.path.isdir(directory_path):
        for img_name in os.listdir(directory_path):
            img_path = os.path.join(directory_path, img_name)
            path_dict[dir_name].append(img_path)

print("Total cat images:", len(path_dict["cats"]))
print("Total dog images:", len(path_dict["dogs"]))

Total cat images: 70
Total dog images: 70


In [19]:
img_size = 64
X = []
y = []

for label, paths in path_dict.items():
    for img_path in paths:
        try:
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (img_size, img_size))
            X.append(img)
            y.append(0 if label == "cats" else 1)  # cats=0, dogs=1
        except:
            pass  # skip corrupted images

X = np.array(X, dtype="float32") / 255.0
y = to_categorical(y)

print("Data shape:", X.shape, y.shape)


Data shape: (140, 64, 64, 3) (140, 2)


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


C:\Users\verma\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 195ms/step - accuracy: 0.4670 - loss: 1.0290 - val_accuracy: 0.5000 - val_loss: 0.6946
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.4996 - loss: 0.6926 - val_accuracy: 0.5000 - val_loss: 0.6928
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.5792 - loss: 0.6841 - val_accuracy: 0.4643 - val_loss: 0.6910
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.7307 - loss: 0.6732 - val_accuracy: 0.5000 - val_loss: 0.6922
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.5607 - loss: 0.6674 - val_accuracy: 0.5000 - val_loss: 0.6951


In [23]:
model_json = model.to_json()
model_weights = model.get_weights()

model_dict = {
    "architecture": model_json,
    "weights": model_weights
}

with open("cat_dog_model.pkl", "wb") as f:
    pickle.dump(model_dict, f)

print("Model saved as cat_dog_model.pkl")


Model saved as cat_dog_model.pkl


In [24]:
with open("cat_dog_model.pkl", "rb") as f:
    loaded_dict = pickle.load(f)

loaded_model = model_from_json(loaded_dict["architecture"])
loaded_model.set_weights(loaded_dict["weights"])
loaded_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("Model loaded from pickle and ready to use.")

Model loaded from pickle and ready to use.


In [25]:
sample_img = X_test[0].reshape(1, img_size, img_size, 3)
pred = loaded_model.predict(sample_img)
print("Prediction (0=Cat, 1=Dog):", np.argmax(pred))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
Prediction (0=Cat, 1=Dog): 1
